In [49]:
import mysql.connector
# library for data analysis
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 

import os 

In [5]:
# Membuat koneksi ke database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='tourista_db'
)

In [76]:
# Mengeksekusi query untuk mengambil data dari tabel

query = "SELECT * FROM destination"
destination = pd.read_sql_query(query, conn)

query = "SELECT * FROM review_wisata"
ratings = pd.read_sql_query(query, conn)

query = "SELECT * FROM user_profile"
users = pd.read_sql_query(query, conn)

C:\Users\dewan\AppData\Local\Temp\ipykernel_10164\707316895.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  destination = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_10164\707316895.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_10164\707316895.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql_query(query, conn)


In [77]:
ratings.head(2)

,user_wisata,wisata_id,wisata_rating
0,1,179,3.0
1,1,344,2.0


In [96]:
users.head(2)

,id,name,age,email,phone_number,address,photo_profile,user_lat,user_lot,createdAt,updatedAt
0,1,user1,20,userTrial1@gmail.com,81326812647,"Semarang, Jawa Tengah",user1.png,-11931.0,12311.0,None,None
1,2,user2,21,userTrial2@gmail.com,81326812647,"Bekasi, Jawa Barat",user2.png,-11931.0,12311.0,None,None


In [99]:
# rename column wisata_id to id 
users = users.rename(columns={'id': 'user_wisata'})

In [100]:
users.head(2)

,user_wisata,name,age,email,phone_number,address,photo_profile,user_lat,user_lot,createdAt,updatedAt
0,1,user1,20,userTrial1@gmail.com,81326812647,"Semarang, Jawa Tengah",user1.png,-11931.0,12311.0,None,None
1,2,user2,21,userTrial2@gmail.com,81326812647,"Bekasi, Jawa Barat",user2.png,-11931.0,12311.0,None,None


In [86]:
# rename column wisata_id to id 
ratings = ratings.rename(columns={'wisata_id': 'id'})

In [87]:
ratings.head(2)

,user_wisata,id,wisata_rating
0,1,179,3.0
1,1,344,2.0


In [124]:
final = pd.merge(ratings, destination, on = 'id',how = 'left')
trial = final[['user_wisata', 'id', 'wisata_rating', 'name_wisata', 'category', 'description_wisata']]
trial.head(2)

,user_wisata,id,wisata_rating,name_wisata,category,description_wisata
0,1,179,3.0,Candi Ratu Boko,Budaya,Situs Ratu Baka atau Candi Boko (Hanacaraka:ê¦...
1,1,344,2.0,Pantai Marina,Bahari,Pantai Marina (bahasa Jawa: ê¦¥ê¦±ê¦¶ê¦±ê¦¶ê¦‚...


In [127]:
# new variable for content based filtering 
place_id = destination['id'].unique().tolist()
place_name = destination['name_wisata'].unique().tolist()
place_category = destination['category'].tolist()

In [145]:
# check the length of each new variable (must have the SAME LENGTH) 
print(f'number of place id = {len(place_id)}')
print(f'number of place name = {len(place_name)}')
print(f'number of place category = {len(place_category)}')

number of place id = 437
number of place name = 437
number of place category = 437


In [144]:
# dictionary for place_recommend
place_recommend = pd.DataFrame({
    'place_id' : place_id,
    'place_name' : place_name,
    'place_category' : place_category
})
place_recommend

,place_id,place_name,place_category
0,1,Monumen Nasional,Budaya
1,2,Kota Tua,Budaya
2,3,Dunia Fantasi,Taman Hiburan
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan
4,5,Atlantis Water Adventure,Taman Hiburan
...,...,...,...
432,433,Museum Mpu Tantular,Budaya
433,434,Taman Bungkul,Taman Hiburan
434,435,Taman Air Mancur Menari Kenjeran,Taman Hiburan
435,436,Taman Flora Bratang Surabaya,Taman Hiburan


In [143]:
# data frame for content based filtering 
data_content = place_recommend
data_content

,place_id,place_name,place_category
0,1,Monumen Nasional,Budaya
1,2,Kota Tua,Budaya
2,3,Dunia Fantasi,Taman Hiburan
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan
4,5,Atlantis Water Adventure,Taman Hiburan
...,...,...,...
432,433,Museum Mpu Tantular,Budaya
433,434,Taman Bungkul,Taman Hiburan
434,435,Taman Air Mancur Menari Kenjeran,Taman Hiburan
435,436,Taman Flora Bratang Surabaya,Taman Hiburan


In [130]:
tfidf= TfidfVectorizer()
tfidf.fit(data_content['place_name'])

TfidfVectorizer()

In [146]:
# change data into matrix 
tfidf_matrix = tfidf.fit_transform(data_content['place_name'])
tfidf_matrix.shape

(437, 712)

In [147]:
# change vector TF-IDF into matrix 
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [148]:
# calculate the cosine similairty between the place name and each entry in 
cosine_similarity = cosine_similarity(tfidf_matrix)
cosine_similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.07709925,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.07709925, 1.        ,
        0.13442662],
       [0.        , 0.        , 0.        , ..., 0.        , 0.13442662,
        1.        ]])

In [149]:
df_cosine_similarity = pd.DataFrame(cosine_similarity, index = data_content['place_name'], columns = data_content['place_name'])
print('Shape : ', df_cosine_similarity.shape)

Shape :  (437, 437)


In [150]:
# recommendation function for content based filtering 

def content_rec(place_name, similarity_data = df_cosine_similarity, items = data_content[['place_name', 'place_category']], k = 5) :
  index = similarity_data.loc[:, place_name].to_numpy().argpartition(
      range(-1, -k, -1)
  )

  closest = similarity_data.columns[index[-1:-(k+2): -1]]
  closest = closest.drop(place_name, errors = 'ignore')

  return pd.DataFrame(closest).merge(items).head(k)

In [151]:
# test the function 
content_rec('Alun-Alun Kota Bandung')

,place_name,place_category
0,Alun Alun Selatan Yogyakarta,Taman Hiburan
1,Alun-alun Utara Keraton Yogyakarta,Budaya
2,Taman Balai Kota Bandung,Taman Hiburan
3,Kota Mini,Taman Hiburan
4,Kota Tua,Budaya
